In [1]:
import pandas as pd
from connection_ms_sql import CreateDatabaseTable as cdt
from read_csv_save_data_ms_sql import ReadCsv as rcsv
from calculation import Calculations as cal
from ploting import Plot as plt
##Read Csv files for train test and ideal data with in the same forlder
server = 'NASIR'  # e.g., 'localhost\SQLEXPRESS' or an IP address
database_name = 'Assignment'  # Connect to the database to create another database or just remove the database from the connection string
username = 'nasir'  # SQL Server username, omit if using Windows authentication
password = 'Cheema01'  # SQL Server password, omit if using Windows authentication
driver  = 'ODBC Driver 17 for SQL Server'
port = '1433'
dataset_path = r'C:\Users\nasir\OneDrive\Desktop\IU-Course Data\PROGRAMMING WITH PYTHON-course2-sem1\3.WRITTEN ASSIGNMENT\written Assignment\Assignment'
file_names = ['train.csv','test.csv','ideal.csv']
tabels_dic = {
            "train_table": [('X', 'FLOAT'), ('Y1 (training func)', 'FLOAT'), ('Y2 (training func)', 'FLOAT'), ('Y3 (training func)', 'FLOAT'), ('Y4 (training func)', 'FLOAT')],
            "test_table": [('X (test func)', 'FLOAT'), ('Y (test func)', 'FLOAT'), ('Delta Y (test func)', 'FLOAT'), ('No. of ideal func', 'VARCHAR(255)')],
            "ideal_table": [('X', 'FLOAT')] + [(f'Y{i} (ideal func)', 'FLOAT') for i in range(1, 51)],
        }
file_to_table_map = {
    'train.csv': 'train_table',
    'test.csv': 'test_table',
    'ideal.csv': 'ideal_table',
}
db_creator = cdt( server, 
                database_name, 
                username, 
                password, 
                driver, 
                port,
                tabels_dic)
db_copy =    rcsv( server, 
                database_name, 
                username, 
                password, 
                driver, 
                port,
                dataset_path,
                file_names,
                tabels_dic,
                file_to_table_map)


db_creator.create_database()
db_creator.create_tables()
#print(db_creator.create_tables())
db_copy.read_csv_to_sql()

# Initialize the engine
db_copy.alchemy_connection()

# Now, you can use db_conn.engine to perform database operations
engine = db_copy.engine

# Example: Reading data into a pandas DataFrame
df_train = pd.read_sql_query("SELECT * FROM train_table", engine)
df_ideal = pd.read_sql_query("SELECT * FROM ideal_table", engine)
df_test = pd.read_sql_query("SELECT * FROM test_table", engine)

# Example usage
#print(df_train.head())
#print(df_ideal.head())
#print(df_test.head())

# Create an instance of Class Calculations
calculations = cal(df_train, df_ideal, df_test)

# Calculate SSD sums and find top four ideal functions
calculations.calculate_criteria1()

# Access the ssd_sums and top_four_ideal_functions directly from the instance
ssd_sums = calculations.get_ssd_sums()
top_four_ideal_functions = calculations.get_top_four_ideal_functions()

#print("SSD sums for all comparisons:", ssd_sums)
#print("Top ideal function for each training function:", top_four_ideal_functions)

# Calculate Daviations
calculations.deviations()
adjusted_deviations = calculations.get_adjusted_deviation()
#Calculate Final Results base on df_test
calculations.results()

#Access Test Resutls
test_results = calculations.get_test_results()

# Create a new DataFrame from the results
df_test_results = pd.DataFrame(test_results)
df_test_results = df_test_results.sort_values(by='X (test func)')

#Write the test Data to test_results table in Microsoft SQL Server
# Initialize the engine
#db_copy.alchemy_connection()

# Now, you can use db_conn.engine to perform database operations
engine = db_copy.engine
table_name = 'test_results'
        
# Save the DataFrame to the SQL table
df_test_results.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
print(f'Data Copied to {table_name} in SQL')

# Create an instance of Class PlotSsdSums
ssd = plt(ssd_sums,df_test_results)

#Plot the DashBoard
ssd.dashboard()
#ssd.ssd_plot_only()
#ssd.scatter_plot_only()


Database "Assignment" already exist
Table 'train_table' Already Present: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'train_table' in the database. (2714) (SQLExecDirectW)")
Table 'test_table' Already Present: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'test_table' in the database. (2714) (SQLExecDirectW)")
Table 'ideal_table' Already Present: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'ideal_table' in the database. (2714) (SQLExecDirectW)")
Data Copied to train_table in SQL
Data Copied to test_table in SQL
Data Copied to ideal_table in SQL
Top ideal function for each training function: ['Y42 (ideal func)', 'Y41 (ideal func)', 'Y11 (ideal func)', 'Y48 (ideal func)']
Data Copied to test_results in SQL
